<a href="https://colab.research.google.com/github/MauricioCastroF/NLP-desafios/blob/main/Desaf%C3%ADo_3_NLP_CastroFMauricio.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Preprocesamiento de texto**
**Bot utilizado de referencia: Rule-Based BOT con TF-IDF**

Objetivo: Generar la correspondencia de texto más similar al texto en castellado contenido en Wikipedia, relacionado con Colombia.

In [1]:
import json
import string
import random
import re # Regular Expressions (regex)
import urllib.request

import numpy as np

# Para leer y parsear el texto en HTML de wikipedia
import bs4 as bs

import nltk
# Descargar el diccionario
nltk.download("punkt")
nltk.download("wordnet")
nltk.download('omw-1.4')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...


True

***El corpus utilizado es wikipedia sobre Colombia en Español:***

In [2]:
raw_html = urllib.request.urlopen('https://es.wikipedia.org/wiki/Colombia')
raw_html = raw_html.read()

# Parsear artículo, 'lxml' es el parser a utilizar
article_html = bs.BeautifulSoup(raw_html, 'lxml')

# Encontrar todos los párrafos del HTML (bajo el tag <p>)
# y tenerlos disponible como lista
article_paragraphs = article_html.find_all('p')

article_text = ''

for para in article_paragraphs:
    article_text += para.text

article_text = article_text.lower()

In [3]:
# Demos un vistazo
article_text

'\n\ncolombia, oficialmente república de colombia, es un país soberano situado en la región noroccidental de américa del sur. se constituye en un estado unitario, social y democrático de derecho cuya forma de gobierno es presidencialista con dos cámaras legislativas. su capital y ciudad más poblada es bogotá.[12]\u200b es una república organizada políticamente en treinta y dos departamentos descentralizados y el distrito capital de bogotá,[13]\u200b sede del gobierno nacional.\nincluyendo la isla de malpelo, el cayo roncador y el banco serrana, el país abarca una superficie de 1 141 748 km²,[3]\u200b por lo que es el vigesimosexto país más grande del mundo y el séptimo más grande de américa. reclama como mar territorial el área hasta las 12 millas náuticas de distancia,[4]\u200b manteniendo un diferendo limítrofe al respecto con venezuela y nicaragua.[14]\u200b[15]\u200b\nlimita al oriente con venezuela y brasil, al sur con perú y ecuador y al occidente con panamá; en cuanto a límites 

***1. Proceso de preprocesamiento***
* Remover caracteres especiales
* Quitar espacios o saltos


In [4]:
# substituir con espacio vacío:
text = re.sub(r'\[[0-9]*\]', ' ', article_text)
text = re.sub(r'\s+', ' ', text)

In [5]:
# Demos un vistazo
text

' colombia, oficialmente república de colombia, es un país soberano situado en la región noroccidental de américa del sur. se constituye en un estado unitario, social y democrático de derecho cuya forma de gobierno es presidencialista con dos cámaras legislativas. su capital y ciudad más poblada es bogotá. \u200b es una república organizada políticamente en treinta y dos departamentos descentralizados y el distrito capital de bogotá, \u200b sede del gobierno nacional. incluyendo la isla de malpelo, el cayo roncador y el banco serrana, el país abarca una superficie de 1 141 748 km², \u200b por lo que es el vigesimosexto país más grande del mundo y el séptimo más grande de américa. reclama como mar territorial el área hasta las 12 millas náuticas de distancia, \u200b manteniendo un diferendo limítrofe al respecto con venezuela y nicaragua. \u200b \u200b limita al oriente con venezuela y brasil, al sur con perú y ecuador y al occidente con panamá; en cuanto a límites marítimos, colinda co

In [6]:
print("Cantidad de caracteres en el texto:", len(text))

Cantidad de caracteres en el texto: 107595


***2. Dividir el texto en sentencias y en palabras***

In [7]:
corpus = nltk.sent_tokenize(text) # divide en oraciones
words = nltk.word_tokenize(text) # divide en términos

In [8]:
# Demos un vistazo
corpus[:10]

[' colombia, oficialmente república de colombia, es un país soberano situado en la región noroccidental de américa del sur.',
 'se constituye en un estado unitario, social y democrático de derecho cuya forma de gobierno es presidencialista con dos cámaras legislativas.',
 'su capital y ciudad más poblada es bogotá.',
 '\u200b es una república organizada políticamente en treinta y dos departamentos descentralizados y el distrito capital de bogotá, \u200b sede del gobierno nacional.',
 'incluyendo la isla de malpelo, el cayo roncador y el banco serrana, el país abarca una superficie de 1 141 748 km², \u200b por lo que es el vigesimosexto país más grande del mundo y el séptimo más grande de américa.',
 'reclama como mar territorial el área hasta las 12 millas náuticas de distancia, \u200b manteniendo un diferendo limítrofe al respecto con venezuela y nicaragua.',
 '\u200b \u200b limita al oriente con venezuela y brasil, al sur con perú y ecuador y al occidente con panamá; en cuanto a lími

In [9]:
# Demos un vistazo
words[:50]

['colombia',
 ',',
 'oficialmente',
 'república',
 'de',
 'colombia',
 ',',
 'es',
 'un',
 'país',
 'soberano',
 'situado',
 'en',
 'la',
 'región',
 'noroccidental',
 'de',
 'américa',
 'del',
 'sur',
 '.',
 'se',
 'constituye',
 'en',
 'un',
 'estado',
 'unitario',
 ',',
 'social',
 'y',
 'democrático',
 'de',
 'derecho',
 'cuya',
 'forma',
 'de',
 'gobierno',
 'es',
 'presidencialista',
 'con',
 'dos',
 'cámaras',
 'legislativas',
 '.',
 'su',
 'capital',
 'y',
 'ciudad',
 'más',
 'poblada']

In [10]:
print("Vocabulario:", len(words))

Vocabulario: 19907


***4. Funciones de ayuda para limpiar y procesar el input del usuario***

* Lematizar los tokens de la oración
* Quitar símbolos de puntuación

In [11]:
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()

def perform_lemmatization(tokens):
    return [lemmatizer.lemmatize(token) for token in tokens]

punctuation_removal = dict((ord(punctuation), None) for punctuation in string.punctuation)

def get_processed_text(document):
    return perform_lemmatization(nltk.word_tokenize(document.lower().translate(punctuation_removal)))

***5. Utilizar vectores TF-IDF y la similitud coseno construido con el corpus del artículo de wikipedia***


In [12]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

def generate_response(user_input, corpus):
    response = ''
    corpus.append(user_input)

    word_vectorizer = TfidfVectorizer(tokenizer=get_processed_text, stop_words='english')

    all_word_vectors = word_vectorizer.fit_transform(corpus)

    similar_vector_values = cosine_similarity(all_word_vectors[-1], all_word_vectors)

    similar_sentence_number = similar_vector_values.argsort()[0][-2]
    matched_vector = similar_vector_values.flatten()
    matched_vector.sort()
    vector_matched = matched_vector[-2]

    if vector_matched == 0: # si la similaridad coseno fue nula (ningún término en común)
        response = "Lo siento, no entendí"
    else:
        response = corpus[similar_sentence_number] # obtener el documento del corpus más similar

    corpus.remove(user_input)
    return response

***6. Ensayar el sistema***
El sistema intentará encontrar la parte del artículo que más se relaciona con nuestro texto de entrada.

In [13]:
import sys
!{sys.executable} -m pip install gradio --quiet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.7/19.7 MB 69.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.0/57.0 kB 8.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 288.4/288.4 kB 34.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.4/75.4 kB 11.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 236.8/236.8 kB 27.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.5/50.5 kB 7.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 137.0/137.0 kB 19.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.7/45.7 kB 6.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 7.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.9/129.9 kB 15.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 6.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.5/46.5 kB 6.5 MB/s eta 0

In [ ]:
import gradio as gr

def bot_response(human_text):
    print("Q:", human_text)
    resp = generate_response(human_text.lower(), corpus)
    print("A:", resp)
    return resp

iface = gr.Interface(
    fn=bot_response,
    inputs=["textbox"],
    outputs="text",
    layout="vertical")

iface.launch(debug=True)

<ipython-input-14-fe9efd385e52>:9: UserWarning: `layout` parameter is deprecated, and it has no effect
  iface = gr.Interface(


Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
Note: opening Chrome Inspector may crash demo inside Colab notebooks.

To create a public link, set `share=True` in `launch()`.


<IPython.core.display.Javascript object>

Q: Bogota


/usr/local/lib/python3.10/dist-packages/sklearn/feature_extraction/text.py:528: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/feature_extraction/text.py:409: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['ha', 'le', 'u', 'wa'] not in stop_words.
  warnings.warn(


A: Lo siento, no entendí
Q: cual es la capital de colombia?
A: su capital y ciudad más poblada es bogotá.
